In [151]:
def remove_null_columns(df):
    mask     = df.isnull().all()
    null     = df.columns[mask]
    not_null = df.columns[np.logical_not(mask)]

    print(f'{len(null)+len(not_null)} colunas \n{len(not_null)} com pelo menos uma informacao \n{len(null)} sem nenhuma informacao')
    display(pd.Series(null))
    
    final_cols = not_null.tolist() + null.tolist()
    
    
    df = df[final_cols]

    return df

def organizer(df,cols):
    
    not_in = [col for col in df.columns.tolist() if col not in cols]
    final_cols = cols + not_in
    df = df[final_cols]
    
    return df

In [152]:
import pandas as pd
import numpy as np


pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot

import os


import requests
import urllib

## Source
 - [Relação de Contratações e Compras / Lei 13.979/2020](https://www.saopaulo.sp.gov.br/coronavirus/transparencia/)


In [153]:
valores = pd.read_csv('../data/orcamento/sp/COVID.csv',sep=';')

#Colunas que tem no portal mas n tem na tabela:  ['Fonte','Local de Entrega']
### Link Nota de Empenho
#http://siafem-appws.fazenda.sp.gov.br/ServicesSiafem/NE_PDF?UG=080358&GESTAO=00001&NE=2020NE00119
#url + Código UG Documento +  + Número Empenho



cols = ['Secretaria','Número Processo','Descrição','Nome Credor','CNPJ CPF','Descrição Processo',
        'Finalidade', 'Quantidade Item','Valor Unitário', 'Valor NE','Data Emissão',
        'Tipo Documento','Número Documento','Valor NP','Valor de Empenho no Processo']

valores = organizer(valores,cols)

In [159]:
valores.head(10)

,Secretaria,Número Processo,Descrição,Nome Credor,CNPJ CPF,Descrição Processo,Finalidade,Quantidade Item,Valor Unitário,Valor NE,Data Emissão,Tipo Documento,Número Documento,Valor NP,Valor de Empenho no Processo,Cod Tot Empenho,Assunto,13979,Nome Órgão Documento,Nome UO Documento,Código UG Documento,Número Empenho,Data Atualização,Código Fonte Recurso,Código Gestão Documento,Sistema Origem NE,Valor NL,Valor OB,Nome Projeto Atividade,Nome Poder
0,Saúde,2020/16884,DISPENSA DE LICITACAO,HICHENS HARRISON CAPITAL PARTNER LLC,EIM number 831426803,AQUISIÇÃO DE EQUIPAMENTO,RESPIRADOR E OUTROS POR IMPORTAÇÃO,1,165247500,165247500,14/04/2020,NE,NaN,0,"242.247.500,00",090102000012020NE00561SIAFEM,Aparelhamento,13.979 Art.4º §2º,SECRETARIA DA SAUDE,ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,90102,2020NE00561,02/06/2020,1,1,SIAFEM,0,0,APARELHAMENTO/EQUIPAMENTOS UNID.ADM.DIR./IND.,PODER EXECUTIVO
1,Saúde,2020/16884,DISPENSA DE LICITACAO,HICHENS HARRISON CAPITAL PARTNER LLC,EIM number 831426803,AQUISIÇÃO DE EQUIPAMENTO,RESPIRADOR E OUTROS POR IMPORTAÇÃO,0,0,0,14/04/2020,NL,2020NL03027,0,"242.247.500,00",090102000012020NE00561SIAFEM,Aparelhamento,13.979 Art.4º §2º,SECRETARIA DA SAUDE,ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,90102,2020NE00561,02/06/2020,1,1,SIAFEM,165247500,0,APARELHAMENTO/EQUIPAMENTOS UNID.ADM.DIR./IND.,PODER EXECUTIVO
2,Saúde,2020/16884,DISPENSA DE LICITACAO,HICHENS HARRISON CAPITAL PARTNER LLC,EIM number 831426803,AQUISIÇÃO DE EQUIPAMENTO,RESPIRADOR E OUTROS POR IMPORTAÇÃO,0,0,0,14/04/2020,OB,2020OB00118,0,"242.247.500,00",090102000012020NE00561SIAFEM,Aparelhamento,13.979 Art.4º §2º,SECRETARIA DA SAUDE,ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,90102,2020NE00561,02/06/2020,1,1,SIAFEM,0,165247500,APARELHAMENTO/EQUIPAMENTOS UNID.ADM.DIR./IND.,PODER EXECUTIVO
3,Saúde,2020/16884,DISPENSA DE LICITACAO,HICHENS HARRISON CAPITAL PARTNER LLC,EIM number 831426803,AQUISIÇÃO DE EQUIPAMENTO,RESPIRADOR E OUTROS POR IMPORTAÇÃO,1,385577500,385577500,23/04/2020,NE,NaN,0,"242.247.500,00",090102000012020NE00584SIAFEM,Aparelhamento,13.979 Art.4º §2º,SECRETARIA DA SAUDE,ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,90102,2020NE00584,02/06/2020,1,1,SIAFEM,0,0,APARELHAMENTO/EQUIPAMENTOS UNID.ADM.DIR./IND.,PODER EXECUTIVO
4,Saúde,2020/16884,DISPENSA DE LICITACAO,HICHENS HARRISON CAPITAL PARTNER LLC,EIM number 831426803,AQUISIÇÃO DE EQUIPAMENTO,RESPIRADOR E OUTROS POR IMPORTAÇÃO,0,0,0,23/04/2020,NL,2020NL03424,0,"242.247.500,00",090102000012020NE00584SIAFEM,Aparelhamento,13.979 Art.4º §2º,SECRETARIA DA SAUDE,ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,90102,2020NE00584,02/06/2020,1,1,SIAFEM,77000000,0,APARELHAMENTO/EQUIPAMENTOS UNID.ADM.DIR./IND.,PODER EXECUTIVO
5,Saúde,2020/16884,DISPENSA DE LICITACAO,HICHENS HARRISON CAPITAL PARTNER LLC,EIM number 831426803,AQUISIÇÃO DE EQUIPAMENTO,RESPIRADOR E OUTROS POR IMPORTAÇÃO,0,0,0,23/04/2020,OB,2020OB00126,0,"242.247.500,00",090102000012020NE00584SIAFEM,Aparelhamento,13.979 Art.4º §2º,SECRETARIA DA SAUDE,ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,90102,2020NE00584,02/06/2020,1,1,SIAFEM,0,77000000,APARELHAMENTO/EQUIPAMENTOS UNID.ADM.DIR./IND.,PODER EXECUTIVO
6,Saúde,2020/16884,DISPENSA DE LICITACAO,HICHENS HARRISON CAPITAL PARTNER LLC,EIM number 831426803,AQUISIÇÃO DE EQUIPAMENTO,RESPIRADOR E OUTROS POR IMPORTAÇÃO,1,-308577500,-308577500,14/05/2020,NE,NaN,0,"242.247.500,00",090102000012020NE00584SIAFEM,Aparelhamento,13.979 Art.4º §2º,SECRETARIA DA SAUDE,ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,90102,2020NE00584,02/06/2020,1,1,SIAFEM,0,0,APARELHAMENTO/EQUIPAMENTOS UNID.ADM.DIR./IND.,PODER EXECUTIVO
7,Educação,20407/20,DISPENSA DE LICITACAO,PICPAY SERVICOS S/A,"2,28964E+13",MERENDA EM CASA,MERENDA EM CASA,1,40280955,40280955,01/04/2020,NE,NaN,0,"121.508.955,00",080358000012020NE00119SIAFEM,Merenda em Casa,13.979 Art.4º §2º,SECRETARIA DA EDUCACAO,COORD.DE INFRAESTRUTURA E SERVICOS ESCOLARES,80358,2020NE00119,02/06/2020,5,1,SIAFEM,0,0,FORNEC. ALIMENTACAO P/ALUNOS EDUCACAO BASICA,PODER EXECUTIVO
8,Educação,20407/20,DISPENSA 

In [154]:
processos = pd.read_csv('../data/orcamento/sp/BaseDados_SPSemPapel.csv',sep=';', encoding='iso-8859-9')

#Colunas que tem no portal mas n tem na tabela:[]
#link processo https://www.saopaulo.sp.gov.br/guia-coronavirus/assets/docs-transparencia/PCSPPRC202000428V01.pdf
#url + Número do Processo + V01.pdf

cols = ['Órgão','Número do Processo','Tipo de Processo','Descrição']

processos = organizer(processos,cols)

In [155]:
contratos = pd.read_csv('../data/orcamento/sp/BaseDados_IMESP.csv',sep=';', encoding='iso-8859-9')

contratos = remove_null_columns(contratos)


#Colunas que tem no portal mas n tem na tabela:[]
cols = ['secretaria','processo','modalidadedescricao','contratado','dataAssinatura',
        'dataPublicacao1','prazocontrato','tipoprazo','statusLicitacao','licitacaoLink']

contratos = organizer(contratos,cols)

72 colunas 
35 com pelo menos uma informacao 
37 sem nenhuma informacao


0                   formalicitacao
1          formalicitacaodescricao
2                    tipolicitacao
3           tipolicitacaodescricao
4                 numeroPublicacao
5                         processo
6                  objetolicitacao
7                       idsegmento
8                descricaoSegmento
9                          idgrupo
10                  descricaoGrupo
11                        idclasse
12                 descricaoClasse
13                   localexecucao
14          localexecucaodescricao
15                 dataPublicacao2
16                 dataPublicacao3
17                  dataHoraEdital
18                dataHoraProposta
19                dataHoraAbertura
20                  dataHoraSessao
21                 dataInscricaoDe
22                dataInscricaoAte
23                  dataHoraLeilao
24                  dataHoraInicio
25                 dataHoraTermino
26                   totalPrevisto
27                            nota
28                  

## CNPJS

In [191]:
file_name = "Relação-de-Compras-COVID19_completa_2804"

df = pd.read_csv('../data/orcamento/csv/Relação-de-Compras-COVID19_completa_2804.csv')
df['fornecedor'] = df['fornecedor'].apply(lambda x: x[:-1] if x[-1]=='.' else x).str.replace('VEDANA COM. DE PROD. E SERVIÇOS PARA SAÚDE EIRELLI-EPP','VEDANA COM. DE PRODUTOS E SERV. P/ SAUDE LTDA')

df['id'] = df['cnpj'].str.replace('.','').str.replace('-','').str.replace('/','')
df['descricao'] = df['descricao'] + '. Valor: ' +  df['valor'].astype(str)
df_values = df.groupby(by=['id','cnpj','fornecedor'], as_index=False).agg({'descricao':'\n\n'.join, 'valor':sum})
df_values = df_values.rename(columns={'cnpj':'CNPJ','valor':'valor total'})

In [192]:
url = f'http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/'

df_final = pd.DataFrame()
for cnpj in df['id'].unique():
    url_csv = url + f'{cnpj}.csv'

    
    try:
        dd = pd.read_csv(url_csv)
        print(url_csv)
        dd = dd.drop(columns= ['Id','CNPJ','Unidade Cadastradora','Ramo do Negócio','Porte da Empresa','CNAE Secundário','Caixa Postal','Recadastrado'])
        dd['id'] = cnpj
        
    except:
        dd = pd.DataFrame([cnpj],columns=['id'])
        
        print(cnpj, 'nao existe')

    
    df_final = pd.concat([df_final,dd],axis=0)

df_final = df_final.sort_values(by='Razão Social')


http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/11034934000160.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/00647935000164.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/04192876000138.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/02456955000183.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/04274988000138.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/04027894000750.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/03951140000133.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/03436772000169.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/94516671000153.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/07118264000193.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/71957310000147.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/48791685000168.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/52202

In [196]:
ddd = pd.merge(df_values, df_final, on='id', how='outer').sort_values(by='Razão Social').drop(columns=['id'])

In [199]:
ddd.to_excel('../data/orcamento/csv/lista_cnpjs.xlsx', index=False)

## Cidades

In [4]:
import pandas as pd
import numpy as np


pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot

import os

## Fonte
 - [Boletim](https://www.sjc.sp.gov.br/servicos/saude/coronavirus/atos-e-despesas/)

In [5]:
#sjc

file_name = "autorizacao-de-fornecimento-covid-ate-24042020"

dd = camelot.read_pdf(f'../data/orcamento/pdf/{file_name}.pdf', pages='all')

In [6]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [10]:
cols = ['material','descricao','quantidade_af','unidade','valor_unitario','valor_total','numero_af','data','modalidade_de_compra','razao_social','unidade_orcamentaria','codigo_dotacao_orcamentaria']
df.columns = cols
df = df[df['material']!='MATERIAL']

df['valor_unitario'] = df['valor_unitario'].apply(lambda x: x.replace('R$','').replace('\n','').strip().replace('.','').replace(',','.'))
df['valor_total'] = df['valor_total'].apply(lambda x: x.replace('R$','').replace('\n','').strip().replace('.','').replace(',','.'))

df = df.replace('\n','')

In [13]:
df.to_csv(f'../data/orcamento/csv/sjc/{file_name}.csv', index=False)

## Fonte
 - [Boletim](http://www.osasco.sp.gov.br/portal-da-transparencia)

In [ ]:
#osasco

file_name = "autorizacao-de-fornecimento-covid-ate-24042020"

dd = camelot.read_pdf(f'../data/orcamento/pdf/{file_name}.pdf', pages='all')

## Taubate

## Fonte
 - [Boletim](http://leideacesso.etransparencia.com.br/taubate.prefeitura.sp/TDAPortalClient.aspx?414)

In [33]:
df = pd.read_excel('../data/orcamento/csv/tbt/contratacoes_taubate.xlsx')

df['cnpj'] = df['Fornecedor'].apply(lambda x: x.split('(')[1].split(')')[0])
df['id'] = df['cnpj'].str.replace('.','').str.replace('-','').str.replace('/','')



df['Finalidade'] = df['Finalidade'] + '. Valor: ' +  df['Despesa Contratada (Empenhada)'].astype(str)
df_values = df.groupby(by=['id','cnpj','Fornecedor'], as_index=False).agg({'Finalidade':'\n\n'.join, 'Despesa Contratada (Empenhada)':sum})
df_values = df_values.rename(columns={'cnpj':'CNPJ','Despesa Contratada (Empenhada)':'valor total'})


In [35]:
url = f'http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/'

df_final = pd.DataFrame()
for cnpj in df['id'].unique():
    url_csv = url + f'{cnpj}.csv'

    
    try:
        dd = pd.read_csv(url_csv)
        print(url_csv)
        dd = dd.drop(columns= ['Id','CNPJ','Unidade Cadastradora','Ramo do Negócio','Porte da Empresa','CNAE Secundário','Caixa Postal','Recadastrado'])
        dd['id'] = cnpj
        
    except:
        dd = pd.DataFrame([cnpj],columns=['id'])
        
        print(cnpj, 'nao existe')

    
    df_final = pd.concat([df_final,dd],axis=0)

df_final = df_final.sort_values(by='Razão Social')


http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/61610283000188.csv
03961840000109 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/61699567000192.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/02190877000118.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/08093976000168.csv
26864805000177 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/67729178000491.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/04063331000121.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/05515873000150.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/55309074000104.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/52202744000192.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/03945035000191.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/07118264000193.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/14112327000160.csv
157254890001

In [36]:
ddd = pd.merge(df_values, df_final, on='id', how='outer').sort_values(by='Razão Social').drop(columns=['id'])

In [38]:
ddd.to_excel('../data/orcamento/csv/tbt/lista_cnpjs_tbt.xlsx', index=False)